## Final Project for IBM Data Science Capstone ##
## Comparison of neighborhoods in Buenos Aires, Argentina and Montevideo, Uruguay ##
## Montevideo ##

Import all the libraries required for the project

In [32]:
import pandas as pd
import numpy as np
# 
import requests
import geocoder
from geopy.geocoders import Nominatim 
#
import matplotlib.cm as cm
import matplotlib.colors as colors
# 
from sklearn.cluster import KMeans
#
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
#
import json 
from pandas.io.json import json_normalize
#
pd.set_option('display.max_rows', 500)

Import Montevideo Neighborhood data

In [66]:
df_mont = pd.read_csv("/Users/davidknight/Downloads/DS-montevideo.csv", sep=',', encoding='maccentraleurope')
df_mont.shape

(61, 3)

Check the shape - not all neighborhoods will have venues in Foursquare

In [34]:
print('The Montevideo dataframe has {} unique neighborhoods.'.format(
        len(df_mont['Neighborhood'].unique())))

The Montevideo dataframe has 61 unique neighborhoods.


Register logon information with Foursquare (Data has been masked).

In [37]:
CLIENT_ID = 'hello' # your Foursquare ID
CLIENT_SECRET = 'how are you?' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


getNearbyVenues takes a dataframe and returns the venues for each row in the dataframe.

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
neighborhood_latitude = df_mont.loc[0, 'Lat'] # neighborhood latitude value
neighborhood_longitude = df_mont.loc[0, 'Long'] # neighborhood longitude value
neighborhood_name = df_mont.loc[0, 'Neighborhood'] # neighborhood name

Build a URL for each call to Foursquare.

In [40]:
LIMIT = 100
#
radius = 1000 # define radius
#
url_mont = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url_mont

'https://api.foursquare.com/v2/venues/explore?&client_id=4O4EX12OQOIHEDWFXUJALPEJ1GUNW14U3UWR0KOXRHBLGICK&client_secret=TRVJAMEIG3SJY41TLPSPPPSUTLMWFNPCFVCMO23BVYKGVRHA&v=20180605&ll=-34.8931,-56.1894&radius=1000&limit=100'

In [41]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Build a list of venues by neighborhood.

In [42]:
mont_venues = getNearbyVenues(names=df_mont['Neighborhood'],
                                   latitudes=df_mont['Lat'],
                                   longitudes=df_mont['Long']
                                  )

Aguada
Aires Puros
Atahualpa
Banados de Carrasco
Barrio Sur
Belvedere
Brazo Oriental
Buceo
Capurro Bella Vista
Carrasco
Carrasco Norte
Casabo Pajas Blancas
Casavalle
Castro Castellanos
Centro
Cerrito
Cerro
Ciudad Vieja
Colon Centro y Noroeste
Colon Sureste Abayuba
Conciliacion
Cordon
Figurita
Flor de MaroĖas
Ituzaingo
Jacinto Vera
Jardines del Hipodromo
La Blanqueada
La Comercial
La Paloma Tomkinson
La Teja
LarraĖaga
Las Acacias
Las Canteras
Lezica Melilla
Malvin
Malvin Norte
Manga
MaroĖas Parque Guarani
Mercado Modelo y Bolivar
Nuevo Paris
Palermo
Parque Rodo
Paso de la Arena
Paso de las Duranas
Pe_arol Lavalleja
Piedras Blancas
Pocitos
Pque Batlle Villa Dolores
Prado Nueva Savona
Punta Carretas
Punta Gorda
Punta Rieles Bella Italia
Reducto
Sayago
Tres Cruces
Tres Ombues Pblo Victoria
Union
Villa EspaĖola
Villa Garcia Manga Rural
Villa MuĖoz Retiro


In [43]:
print('The Montevideo dataframe has {} unique neighborhoods.'.format(
        len(mont_venues['Neighborhood'].unique())))

The Montevideo dataframe has 49 unique neighborhoods.


Count Venue Categories.

In [78]:
mont_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,1,1,1,1,1,1
American Restaurant,3,3,3,3,3,3
Art Gallery,1,1,1,1,1,1
Art Museum,3,3,3,3,3,3
Arts & Crafts Store,1,1,1,1,1,1
Athletics & Sports,5,5,5,5,5,5
Auto Garage,1,1,1,1,1,1
BBQ Joint,20,20,20,20,20,20
Bakery,15,15,15,15,15,15


Count venues by neighborhood.

In [48]:
mont_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aguada,10,10,10,10,10,10
Aires Puros,3,3,3,3,3,3
Atahualpa,4,4,4,4,4,4
Barrio Sur,12,12,12,12,12,12
Belvedere,1,1,1,1,1,1
Brazo Oriental,4,4,4,4,4,4
Buceo,28,28,28,28,28,28
Capurro Bella Vista,10,10,10,10,10,10
Carrasco,5,5,5,5,5,5


In [49]:
print('There are {} uniques categories.'.format(len(mont_venues['Venue Category'].unique())))

There are 135 uniques categories.


Add one hot encoding.

In [50]:
# one hot encoding
mont_onehot = pd.get_dummies(mont_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mont_onehot['Neighborhood'] = mont_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mont_onehot.columns[-1]] + list(mont_onehot.columns[:-1])
mont_onehot = mont_onehot[fixed_columns]

# mont_onehot.head()

In [21]:
mont_onehot.shape

(544, 136)

In [51]:
mont_grouped = mont_onehot.groupby('Neighborhood').mean().reset_index()
# mont_grouped

In [23]:
mont_grouped.shape

(49, 136)

Show top 5 venues by neighborhood

In [52]:
num_top_venues = 5

for hood in mont_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mont_grouped[mont_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aguada----
                      venue  freq
0                Restaurant   0.2
1  Mediterranean Restaurant   0.1
2                     Plaza   0.1
3         Food & Drink Shop   0.1
4         Convenience Store   0.1


----Aires Puros----
                   venue  freq
0                 Garden  0.33
1  General Entertainment  0.33
2                 Lounge  0.33
3               Pharmacy  0.00
4              Nightclub  0.00


----Atahualpa----
            venue  freq
0       Gastropub  0.25
1  Ice Cream Shop  0.25
2    Soccer Field  0.25
3          Bakery  0.25
4        Pharmacy  0.00


----Barrio Sur----
                venue  freq
0                 Bar  0.25
1  Basketball Stadium  0.08
2         Coffee Shop  0.08
3            Wine Bar  0.08
4  Athletics & Sports  0.08


----Belvedere----
                 venue  freq
0        Women's Store   1.0
1               Museum   0.0
2  Rental Car Location   0.0
3     Recording Studio   0.0
4           Public Art   0.0


----Brazo Oriental----
 

Generic return_most_common_venues function.

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

top 10 venues

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mont_grouped['Neighborhood']

for ind in np.arange(mont_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mont_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(49)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aguada,Restaurant,Deli / Bodega,Mediterranean Restaurant,Plaza,Train Station,Convenience Store,Mobile Phone Shop,BBQ Joint,Food & Drink Shop,Football Stadium
1,Aires Puros,General Entertainment,Lounge,Garden,Women's Store,Food & Drink Shop,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court
2,Atahualpa,Gastropub,Soccer Field,Bakery,Ice Cream Shop,Diner,Dive Bar,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Flea Market
3,Barrio Sur,Bar,Pub,Comfort Food Restaurant,Steakhouse,Basketball Stadium,Bakery,Coffee Shop,Athletics & Sports,Wine Bar,South American Restaurant
4,Belvedere,Women's Store,Garden,Diner,Dive Bar,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop
5,Brazo Oriental,Gym,Bus Station,BBQ Joint,Breakfast Spot,Women's Store,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court
6,Buceo,Clothing Store,Hotel,Women's Store,Sporting Goods Shop,Pizza Place,Coffee Shop,Soccer Field,South American Restaurant,Café,Shopping Mall
7,Capurro Bella Vista,Bakery,Food & Drink Shop,BBQ Joint,Nightclub,Basketball Court,Ice Cream Shop,Pizza Place,Plaza,Fast Food Restaurant,Flea Market
8,Carrasco,Deli / Bodega,Tea Room,American Restaurant,Art Museum,Pharmacy,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court
9,Carrasco Norte,Bakery,Park,Deli / Bodega,Bus Station,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Women's Store


## Clustering ## 

In [58]:
# set number of clusters
kclusters = 9

mont_grouped_clustering = mont_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mont_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 0, 0, 0], dtype=int32)

In [60]:
# add clustering labels
#
# ** UNcomment This!!  **
#
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mont_merged = df_mont

# merge ba_grouped with ba_data to add latitude/longitude for each neighborhood
mont_merged = mont_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how="inner")

Create a map using Folium.

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mont_merged['Lat'], mont_merged['Long'], mont_merged['Neighborhood'], mont_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
map_clusters.save("/Users/davidknight/Downloads/montevideo_clusters.html")

Generate reports by cluster showing most common venues.

In [77]:
mont_merged.loc[mont_merged['Cluster Labels'] == 0, mont_merged.columns[[0] + list(range(3, mont_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aguada,0,Restaurant,Deli / Bodega,Mediterranean Restaurant,Plaza,Train Station,Convenience Store,Mobile Phone Shop,BBQ Joint,Food & Drink Shop,Football Stadium
1,Aires Puros,0,General Entertainment,Lounge,Garden,Women's Store,Food & Drink Shop,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court
2,Atahualpa,0,Gastropub,Soccer Field,Bakery,Ice Cream Shop,Diner,Dive Bar,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Flea Market
4,Barrio Sur,0,Bar,Pub,Comfort Food Restaurant,Steakhouse,Basketball Stadium,Bakery,Coffee Shop,Athletics & Sports,Wine Bar,South American Restaurant
6,Brazo Oriental,0,Gym,Bus Station,BBQ Joint,Breakfast Spot,Women's Store,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court
7,Buceo,0,Clothing Store,Hotel,Women's Store,Sporting Goods Shop,Pizza Place,Coffee Shop,Soccer Field,South American Restaurant,Café,Shopping Mall
8,Capurro Bella Vista,0,Bakery,Food & Drink Shop,BBQ Joint,Nightclub,Basketball Court,Ice Cream Shop,Pizza Place,Plaza,Fast Food Restaurant,Flea Market
9,Carrasco,0,Deli / Bodega,Tea Room,American Restaurant,Art Museum,Pharmacy,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court
10,Carrasco Norte,0,Bakery,Park,Deli / Bodega,Bus Station,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Women's Store
12,Casavalle,0,Convenience Store,Taxi Stand,Women's Store,Food & Drink Shop,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Food


In [45]:
mont_merged.loc[mont_merged['Cluster Labels'] == 1, mont_merged.columns[[0] + list(range(3, mont_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Reducto,1,Gun Range,Women's Store,Food & Drink Shop,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Food,Garden


In [46]:
mont_merged.loc[mont_merged['Cluster Labels'] == 2, mont_merged.columns[[0] + list(range(3, mont_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Belvedere,2,Women's Store,Garden,Diner,Dive Bar,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop


In [47]:
mont_merged.loc[mont_merged['Cluster Labels'] == 3, mont_merged.columns[[0] + list(range(3, mont_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Castro Castellanos,3,Soccer Field,Grocery Store,Pizza Place,Women's Store,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food,Fast Food Restaurant
30,La Teja,3,Soccer Field,Women's Store,Food,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Flea Market
32,Las Acacias,3,Food Truck,Soccer Field,Women's Store,Food,Fried Chicken Joint,Fountain,Football Stadium,Food Court,Food & Drink Shop,Flea Market


In [48]:
mont_merged.loc[mont_merged['Cluster Labels'] == 4, mont_merged.columns[[0] + list(range(3, mont_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Ituzaingo,4,Bakery,Women's Store,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food,Garden


In [49]:
mont_merged.loc[mont_merged['Cluster Labels'] == 5, mont_merged.columns[[0] + list(range(3, mont_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Las Canteras,5,Pharmacy,Food,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Women's Store,Irish Pub
56,Tres Ombues Pblo Victoria,5,Pharmacy,Food,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Women's Store,Irish Pub


In [50]:
mont_merged.loc[mont_merged['Cluster Labels'] == 6, mont_merged.columns[[0] + list(range(3, mont_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Paso de la Arena,6,Beach,Women's Store,Food & Drink Shop,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Food,Garden


In [51]:
mont_merged.loc[mont_merged['Cluster Labels'] == 7, mont_merged.columns[[0] + list(range(3, mont_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Colon Centro y Noroeste,7,Public Art,Historic Site,Women's Store,Food,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Fast Food Restaurant
44,Paso de las Duranas,7,History Museum,Historic Site,Women's Store,Food,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Fast Food Restaurant


In [52]:
mont_merged.loc[mont_merged['Cluster Labels'] == 8, mont_merged.columns[[0] + list(range(3, mont_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,Nuevo Paris,8,Pizza Place,Women's Store,Food,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Flea Market
58,Villa Espa§ola,8,Convenience Store,Pizza Place,Women's Store,Food & Drink Shop,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Flea Market
